In [1]:
# imports
import time
from globus_compute_sdk import Client, Executor
from globus_compute_sdk.serialize import CombinedCode

#### Set the endpoint ID to use

You should replace the `example_endpoint_id` value in the cell below with the value you got from authenticating to the Endpoint running in the Docker container on your system. Alternatively, you can use the publicly-available "`tutorial_endpoint_id`" but it's much less reliable.

In [2]:
# Public tutorial endpoint
tutorial_endpoint_id = "4b116d3c-1703-4f8f-9f6f-39921e5864df"

# ID of endpoint running in a Docker container (replace this with the ID you get from
# running the "globus-compute-endpoint start example_endpoint" in the Docker container)
example_endpoint_id = "3baaffbd-0378-4eff-8a5b-6243c7f717d5"  # REPLACE THIS

ENDPOINT_ID = example_endpoint_id  # tutorial_endpoint_id

#### Create the Client and Executor

In [3]:
gc = Client(code_serialization_strategy=CombinedCode())

# Get some information about the endpoint
print(f"Endpoint status: {gc.get_endpoint_status(ENDPOINT_ID)}")

gce = Executor(endpoint_id=ENDPOINT_ID, client=gc)

Endpoint status: {'details': {'total_workers': 12, 'idle_workers': 12, 'pending_tasks': 0, 'outstanding_tasks': 0, 'managers': 1, 'active_managers': 1, 'new_core_hrs': 0, 'scheduler_mode': 0, 'rp_processed_timestamp': '1718918515.797614', 'min_blocks': 0, 'total_core_hrs': 0, 'worker_mode': 0, 'cores_per_worker': 1.0, 'scaling_enabled': True, 'max_blocks': 1, 'nodes_per_block': 1, 'prefetch_capacity': 0, 'heartbeat_period': 30, 'mem_per_worker': None, 'total_mem': 0, 'max_workers_per_node': 'Infinity', 'total_cores': 0}, 'status': 'online'}


#### "Hello World" example

In [4]:
def hello_world():
    "A simple 'hello world' function to demonstrate running FaaS with Globus Compute"
    return "Hello World!"


print("Submitting 'hello world' task")
future = gce.submit(hello_world)

while not future.done():
    print("waiting 2 secs for job to finish running")
    time.sleep(2)

print("Result available:")
print(future.result())

Submitting 'hello world' task
waiting 2 secs for job to finish running
Result available:
Hello World!


#### Catching Errors

In [5]:
def division_by_zero():
    return 5 / 0  # This will raise a ZeroDivisionError


future = gce.submit(division_by_zero)

try:
    future.result()
except Exception as exc:
    print(f"Globus Compute returned an exception: {exc}")

Globus Compute returned an exception: 
 Traceback (most recent call last):
   File "/var/folders/d3/99xlftrn42b1_dy400jkmlpr0000gn/T/ipykernel_77878/3911509638.py", line 117, in division_by_zero
 ZeroDivisionError: division by zero



#### Adding arguments
You can add simple arguments to function calls. They must be serializable using the standard Python "Dill" library, and they must be small in size

In [6]:
def get_sum(a, b):
    return a + b

future = gce.submit(get_sum, 10, 5)
print(f"10 + 5 = {future.result()}")

10 + 5 = 15


#### Using dependencies in functions
When you submit functions to run, any dependencies must be imported in the body of the function, and they must be installed in the Python environment that's running on the endpoint.

In [7]:
def get_date():
    from datetime import date
    return date.today()

future = gce.submit(get_date)

print(f"Date fetched from endpoint: {future.result()}")

Date fetched from endpoint: 2024-06-20


In [8]:
def get_random_numpy_array_mean():
    """ This function will throw an error because numpy isn't installed in the Python
    environment on the remote endpoint
    """
    import numpy as np
    random_array = np.random.rand(100, 100)
    return np.mean(random_array)

future = gce.submit(get_random_numpy_array_mean)

try:
    print(f"Mean of random array = {future.result()}")
except Exception as exc:
    print(f"Globus Compute returned an exception: {exc}")

Globus Compute returned an exception: 
 Traceback (most recent call last):
   File "/var/folders/d3/99xlftrn42b1_dy400jkmlpr0000gn/T/ipykernel_77878/3799816297.py", line 6, in get_random_numpy_array_mean
 ModuleNotFoundError: No module named 'numpy'



#### Running external processes
You can also run external processes on the Endpoint

In [9]:
def print_uname():
    """ This will return "root" because the Python process on the endpoint is running as
    the "root" user in the Docker container
    """
    import os
    return os.popen("whoami").read()

future = gce.submit(print_uname)

print("User on remote Endpoint: ", future.result())

User on remote Endpoint:  root

